In [1]:
import os
from glob import glob

import dask
import numpy as np
import xarray as xr

## Preliminaries

In [2]:
################
#### Paths #####
################
# Update these for reproduction

project_data_path = "/storage/group/pches/default/users/dcl5300/conus_comparison_lafferty-etal-2024/"
project_code_path = "/storage/home/dcl5300/work/current_projects/conus_comparison_lafferty-etal-2024/"
loca_path = "/storage/group/pches/default/public/LOCA2" # raw loca outputs

In [3]:
##############
### Models ###
##############

from utils import loca_all
models = list(loca_all.keys())

# Matches website (https://loca.ucsd.edu/loca-version-2-for-north-america-ca-jan-2023/) as of Jan 2023
print(f"# models: {len(models)}")
print(f"# model/expts: {np.sum([len(loca_all[model]) for model in models])}")
print(f"# model/expts/ens: {np.sum([len(loca_all[model][ssp]) for model in models for ssp in loca_all[model]])}")

# models: 27
# model/expts: 99
# model/expts/ens: 329


In [4]:
## File path function
def make_loca_file_path(loca_path, model, member, ssp, var):
    """
    Returns list of file paths for a given downscaled LOCA output.
    """
    out_path = f"{loca_path}/{model}/0p0625deg/{member}/{ssp}/{var}"

    if os.path.isdir(out_path):
        files = os.listdir(out_path)
        files = [file for file in files if file[-7:] != 'ORIG.nc'] # Skip ORIGs (had to fix tasmin naming errors)
        return files
    else:
        return []

In [18]:
###############################
# Metric calulcation function #
###############################
def calculate_metric(model, member, ssp, var, metric, loca_path, out_path):
    """
    Inputs: selected model, member, ssp, variable, and metric to calculate (from LOCA)
    Outputs: calculated (annual) metric (max, avg, sum)
    """ 
    # Get files
    if var == "tas":
        files = make_loca_file_path(loca_path, model, member, ssp, "tasmin")
    else: 
        files = make_loca_file_path(loca_path, model, member, ssp, var)
            
    ## Loop through files
    for file in files:
        try:
            ## First check if already exists
            if var == "tas":
                out_str = file.replace("tasmin", "tas")
            else:
                out_str = file
            if os.path.isfile(f"{out_path}/{metric}_{out_str}"):
                continue

            # Read & calculate
            if var == "tas":
                file_path = f"{loca_path}/{model}/0p0625deg/{member}/{ssp}/tasmin/{file}"
                
                tasmin_file_path = file_path
                tasmax_file_path = file_path.replace("tasmin", "tasmax")
                
                ds_tasmin_tmp = xr.open_dataset(tasmin_file_path)
                ds_tasmax_tmp = xr.open_dataset(tasmax_file_path)
                
                ds_tmp = xr.merge([ds_tasmin_tmp, ds_tasmax_tmp],
                                  combine_attrs = "drop_conflicts")
                        
                ds_tmp["tas"] = (ds_tmp["tasmin"] + ds_tmp["tasmax"]) / 2.0
                
                ds_tmp = ds_tmp.drop_vars(["tasmin","tasmax"])
            else:
                file_path = loca_path + model + "/0p0625deg/" + member + "/" + ssp + "/" + var + "/" + file
                ds_tmp = xr.open_dataset(file_path)
            
            # Convert units
            if var in ["tas", "tasmax", "tasmin"]:
                ds_tmp[var] = ds_tmp[var] - 273.15    
                ds_tmp[var].attrs["units"] = "C"
            elif var == "pr":
                ds_tmp[var] = ds_tmp[var] * 86400
                ds_tmp[var].attrs["units"] = "mm/day"
    
            # Calculate metric
            if metric == "avg":
                ds_out = ds_tmp.resample(time="1Y").mean()
            elif metric == "max":
                ds_out = ds_tmp.resample(time="1Y").max()
            elif metric == "sum":
                ds_out = ds_tmp.resample(time="1Y").sum()
                if var == "pr":
                    ds_out.pr.attrs["units"] = "mm"
            
            del ds_tmp
                        
            # Store
            if var == "tas":
                out_str = file.replace("tasmin", "tas")
            else:
                out_str = file
                
            ds_out.to_netcdf(f"{out_path}/{metric}_{out_str}")
            
        # Log if error
        except Exception as e:
            except_path = f"{project_code_path}/code/logs"
            with open(f"{except_path}/{model}_{member}_{ssp}_{var}_LOCA.txt", "w") as f:
                f.write(str(e))

In [6]:
############
### Dask ###
############
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    account="pches",
    # account="open",
    cores=1,
    memory="40GiB",
    walltime="00:30:00"
)

cluster.scale(jobs=30)  # ask for jobs

from dask.distributed import Client

client = Client(cluster)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.0.162:40743,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Calculate metrics

In [23]:
%%time
#########################
## Average Temperature ##
#########################
var = "tas"
metric = "avg"

out_path = f"{project_data_path}/metrics/LOCA2/"

# Parallelize over dask delayed
delayed = []

# Loop through models
for model in models:
    # Loop through SSPs
    for ssp in list(loca_all[model].keys()):
        # Loop through members
        for member in loca_all[model][ssp]:
            # Some vars are missing for some outputs: skip
            file_paths = make_loca_file_path(loca_path, model, member, ssp, "tasmin")
            if len(file_paths) == 0:
                print(f"{model} {ssp} {member}")
                    
            # Calculate metric
            delayed.append(dask.delayed(calculate_metric)(model = model,
                                                          member = member,
                                                          ssp = ssp,
                                                          var = var,
                                                          metric = metric,
                                                          loca_path = loca_path,
                                                          out_path = out_path))
                
# Compute
print(f"# computations: {len(delayed)} \n")
out = dask.compute(*delayed)

MPI-ESM1-2-LR ssp585 r5i1p1f1
MPI-ESM1-2-LR ssp585 r6i1p1f1
MPI-ESM1-2-LR ssp585 r7i1p1f1
MPI-ESM1-2-LR ssp585 r8i1p1f1
MPI-ESM1-2-LR ssp585 r10i1p1f1
# computations: 329 

CPU times: user 260 ms, sys: 22.2 ms, total: 283 ms
Wall time: 496 ms


In [15]:
%%time
#########################
## Total Precipitation ##
#########################
var = "pr"
metric = "sum"

out_path = f"{project_data_path}/metrics/LOCA2/"

# Parallelize over dask delayed
delayed = []

# Loop through models
for model in models:
    # Loop through SSPs
    for ssp in list(loca_all[model].keys()):
        # Loop through members
        for member in loca_all[model][ssp]:
            # Some vars are missing for some outputs: skip
            file_paths = make_loca_file_path(loca_path, model, member, ssp, var)
            if len(file_paths) == 0:
                print(f"{model} {ssp} {member}")
                    
            # Calculate metric
            delayed.append(dask.delayed(calculate_metric)(model = model,
                                                          member = member,
                                                          ssp = ssp,
                                                          var = var,
                                                          metric = metric,
                                                          loca_path = loca_path,
                                                          out_path = out_path))
                
# Compute
print(f"# computations: {len(delayed)} \n")
_ = dask.compute(*delayed)

# computations: 329 

CPU times: user 294 ms, sys: 43.1 ms, total: 337 ms
Wall time: 489 ms


In [21]:
%%time
#########################
## Maximum Temperature ##
#########################
var = "tasmax"
metric = "max"

out_path = f"{project_data_path}/metrics/LOCA2/"

# Parallelize over dask delayed
delayed = []

# Loop through models
for model in models:
    # Loop through SSPs
    for ssp in list(loca_all[model].keys()):
        # Loop through members
        for member in loca_all[model][ssp]:
            # Some vars are missing for some outputs: skip
            file_paths = make_loca_file_path(loca_path, model, member, ssp, var)
            if len(file_paths) == 0:
                print(f"{model} {ssp} {member}")
                    
            # Calculate metric
            delayed.append(dask.delayed(calculate_metric)(model = model,
                                                          member = member,
                                                          ssp = ssp,
                                                          var = var,
                                                          metric = metric,
                                                          loca_path = loca_path,
                                                          out_path = out_path))
                
                
# Compute           
print(f"# computations: {len(delayed)} \n")
out = dask.compute(*delayed)

MPI-ESM1-2-LR ssp585 r5i1p1f1
MPI-ESM1-2-LR ssp585 r6i1p1f1
MPI-ESM1-2-LR ssp585 r7i1p1f1
MPI-ESM1-2-LR ssp585 r8i1p1f1
MPI-ESM1-2-LR ssp585 r10i1p1f1
# computations: 329 

CPU times: user 625 ms, sys: 64.3 ms, total: 689 ms
Wall time: 1.94 s


In [19]:
%%time
#########################
# Maximum Precipitation #
#########################
var = "pr"
metric = "max"

out_path = f"{project_data_path}/metrics/LOCA2/"

# Parallelize over dask delayed
delayed = []

# Loop through models
for model in models:
    # Loop through SSPs
    for ssp in list(loca_all[model].keys()):
        # Loop through members
        for member in loca_all[model][ssp]:
            # Some vars are missing for some outputs: skip
            file_paths = make_loca_file_path(loca_path, model, member, ssp, var)
            if len(file_paths) == 0:
                print(f"{model} {ssp} {member}")
                    
            # Calculate metric
            delayed.append(dask.delayed(calculate_metric)(model = model,
                                                          member = member,
                                                          ssp = ssp,
                                                          var = var,
                                                          metric = metric,
                                                          loca_path = loca_path,
                                                          out_path = out_path))
                
# Compute           
print(f"# computations: {len(delayed)} \n")
out = dask.compute(*delayed)

# computations: 329 

CPU times: user 322 ms, sys: 43.2 ms, total: 365 ms
Wall time: 572 ms
